In [1]:
## Consider database of fruits

fruits = {
    'Merry': [
        ('apple', 5),
        ('orange', 3),
    ],
    'John': [
        ('berries', 1),
        ('orange', 10),
    ],
    'Tom' : [
        ('peaches', 2),
    ],
}
fruits.items()

[('John', [('berries', 1), ('orange', 10)]),
 ('Merry', [('apple', 5), ('orange', 3)]),
 ('Tom', [('peaches', 2)])]

In [ ]:
filter(fruits, lambda: (key in))